In [2]:
#Data analysis
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
# from sklearn_crfsuite import CRF, scorers, metrics
# import sklearn_crfsuite
# from sklearn_crfsuite import scorers
# from sklearn_crfsuite import metrics
# from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import scipy.stats
import nltk
import sklearn_crfsuite
# import eli5
import os
os.chdir("/scratch/spanchak/SWM")

In [3]:
import re
import ast

In [3]:
print(os.listdir())

['y_train.json', 'test_sample.csv', 'NER_tagged_data.csv', 'train.csv', 'X_train.json', 'test.csv', 'train_sample.csv']


In [4]:
data = pd.read_csv("NER_tagged_data.csv")

In [5]:
data.shape

(1204518, 5)

In [6]:
data.head()

,tweet_id,tweet_text,tweet_type,language,NER_tagged_data
0,1661792408139431937,"Valencia player, diakhaby didn't pose in the p...",retweet,en,"[('Valencia', 'PERSON'), ('player', 'NON_NER')..."
1,1661829914817863717,"Self-proclaimed antifa member & researcher, di...",retweet,en,"[('Self', 'NON_NER'), ('-', 'NON_NER'), ('proc..."
2,1661224704567308289,"I cannot trust self-proclaim ""anti-war russian...",retweet,en,"[('I', 'NON_NER'), ('can', 'NON_NER'), ('not',..."
3,1661702658506489857,We have seen you people in action and you all ...,reply,en,"[('We', 'NON_NER'), ('have', 'NON_NER'), ('see..."
4,1661672242391187457,"We don't know about others, but our champ, is ...",retweet,en,"[('We', 'NON_NER'), ('do', 'NON_NER'), (""n't"",..."


In [7]:
'''
train test split
sample data
'''

'\ntrain test split\nsample data\n'

In [8]:
train = data.sample(frac=0.7)
test = data.merge(train,indicator = True, how='left').loc[lambda x : x['_merge']!='both']

In [9]:
train.shape

(843163, 5)

In [10]:
test.shape

(361355, 6)

In [11]:
train.to_csv("train.csv",index=False)

In [12]:
test.to_csv("test.csv",index=False)

In [13]:
# train_sample = train.sample(frac=0.01)
# test_sample = test.sample(frac=0.01)

In [14]:
# train_sample.to_csv("train_sample.csv",index=False)
# test_sample.to_csv("test_sample.csv",index=False)

In [15]:
# for i in train_sample['NER_tagged_data'].to_list()[:5]:
#     print(i,end="\n\n")

In [16]:
def convert_to_list(example_string):
    # try:
    #     # Define the regular expression pattern
    #     pattern = r'\((.*?)\)'
        
    #     # Use re.findall to find all matches in the input string
    #     matches = re.findall(pattern, string)
        
    #     # Print the matches
    #     # print(matches[0])
    #     matches = [i.strip().split(",") for i in matches]
    #     print(matches)
    #     [print(i,end="\n\n") for i in matches if len(i)<2]
    #     matches = [(i[0].strip()[1:-1], i[1].strip()[1:-1]) for i in matches]
    #     return matches
    # except:
    #     print(string)
    #     return []

    example_string = example_string.replace('\n', '')

    # Convert the string to a list of tuples using ast.literal_eval
    list_of_tuples = ast.literal_eval(example_string)
    return list_of_tuples

In [17]:
# print(train_sample['NER_tagged_data'].to_list()[1])
# convert_to_list(train_sample['NER_tagged_data'].to_list()[1])

In [18]:


# # Your example string
# example_string = '''[('On', 'NON_NER'), ('this', 'DATE'), ('day', 'DATE'), ('in', 'NON_NER'), ('1921', 'DATE'), (',', 'NON_NER'), ('the', 'NON_NER'), ('tulsa', 'NON_NER'), ('race', 'NON_NER'), ('massacre', 'NON_NER'), ('happened', 'NON_NER'), ('in', 'NON_NER'), ('the', 'NON_NER'), ('affluent', 'NON_NER'), ('black', 'NON_NER'), ('community', 'NON_NER'), ('of', 'NON_NER'), ('greenwood', 'NON_NER'), ('in', 'NON_NER'), ('tulsa', 'NON_NER'), ('(', 'NON_NER'), ('black', 'NON_NER'), ('wall', 'NON_NER'), ('street', 'NON_NER'), (')', 'NON_NER'), ('\n', 'NON_NER'), ('white', 'NON_NER'), ('supremacists', 'NON_NER'), ('killed', 'NON_NER'), ('more', 'CARDINAL'), ('than', 'CARDINAL'), ('300', 'CARDINAL'), ('african', 'NORP'), ('americans', 'NORP'), ('and', 'NON_NER'), ('looted', 'NON_NER'), ('&', 'NON_NER'), ('burned', 'NON_NER'), ('to', 'NON_NER'), ('ground', 'NON_NER'), ('homes', 'NON_NER'), ('&', 'NON_NER'), ('businesses', 'NON_NER'), ('.', 'NON_NER'), ('\n', 'NON_NER'), ('history', 'NON_NER'), ('of', 'NON_NER'), ('tulsa', 'NON_NER'), ('before', 'NON_NER'), ('the', 'NON_NER'), ('riot', 'NON_NER'), ('\n', 'NON_NER'), ('a', 'NON_NER'), ('thread', 'NON_NER')]'''
# example_string = example_string.replace('\n', '')

# # Convert the string to a list of tuples using ast.literal_eval
# list_of_tuples = ast.literal_eval(example_string)

# # Print the result
# print(list_of_tuples)

In [19]:
# convert_to_list('''[('On', 'NON_NER'), ('this', 'DATE'), ('day', 'DATE'), ('in', 'NON_NER'), ('1921', 'DATE'), (',', 'NON_NER'), ('the', 'NON_NER'), ('tulsa', 'NON_NER'), ('race', 'NON_NER'), ('massacre', 'NON_NER'), ('happened', 'NON_NER'), ('in', 'NON_NER'), ('the', 'NON_NER'), ('affluent', 'NON_NER'), ('black', 'NON_NER'), ('community', 'NON_NER'), ('of', 'NON_NER'), ('greenwood', 'NON_NER'), ('in', 'NON_NER'), ('tulsa', 'NON_NER'), ('(', 'NON_NER'), ('black', 'NON_NER'), ('wall', 'NON_NER'), ('street', 'NON_NER'), (')', 'NON_NER'), ('\n', 'NON_NER'), ('white', 'NON_NER'), ('supremacists', 'NON_NER'), ('killed', 'NON_NER'), ('more', 'CARDINAL'), ('than', 'CARDINAL'), ('300', 'CARDINAL'), ('african', 'NORP'), ('americans', 'NORP'), ('and', 'NON_NER'), ('looted', 'NON_NER'), ('&', 'NON_NER'), ('burned', 'NON_NER'), ('to', 'NON_NER'), ('ground', 'NON_NER'), ('homes', 'NON_NER'), ('&', 'NON_NER'), ('businesses', 'NON_NER'), ('.', 'NON_NER'), ('\n', 'NON_NER'), ('history', 'NON_NER'), ('of', 'NON_NER'), ('tulsa', 'NON_NER'), ('before', 'NON_NER'), ('the', 'NON_NER'), ('riot', 'NON_NER'), ('\n', 'NON_NER'), ('a', 'NON_NER'), ('thread', 'NON_NER')]''')

In [20]:
# train_sample['sent_list'] = train_sample['NER_tagged_data'].apply(convert_to_list)
# test_sample['sent_list'] = test_sample['NER_tagged_data'].apply(convert_to_list)

In [21]:
train['sent_list'] = train['NER_tagged_data'].apply(convert_to_list)
test['sent_list'] = test['NER_tagged_data'].apply(convert_to_list)

In [22]:
def word2features(sent, i):
    word = sent[i][0]
    # postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        # 'postag': postag,
        # 'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            # '-1:postag': postag1,
            # '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            # '+1:postag': postag1,
            # '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [23]:
# X_train[0][1]

In [24]:
X_train = [sent2features(s) for s in train['sent_list'].to_list()]

In [1]:
import json

In [26]:
# feat_dict = json.dumps()

In [27]:
with open("X_train.json","w") as file:
    json.dump({"X_train":X_train},file)

In [28]:
y_train = [sent2labels(s) for s in train['sent_list'].to_list()]

In [29]:
with open("y_train.json","w") as file:
    json.dump({"y_train":y_train},file)

In [ ]:
X_test = [sent2features(s) for s in test['sent_list'].to_list()]

In [ ]:
with open("X_test.json","w") as file:
    json.dump({"X_test":X_test},file)

In [ ]:
y_test = [sent2labels(s) for s in test['sent_list'].to_list()]

In [ ]:
with open("y_test.json","w") as file:
    json.dump({"y_test":y_test},file)

In [5]:
with open("X_train.json","r") as file:
    feat = json.load(file)

In [7]:
len(feat["X_train"])

843163

In [8]:
with open("y_train.json","r") as file:
    label = json.load(file)

In [11]:
len(label["y_train"])

843163

In [12]:
data = pd.DataFrame()
data['feat'] = feat["X_train"]
data['label'] = label["y_train"]

In [14]:
data.head()

,feat,label
0,"[{'bias': 1.0, 'word.lower()': 'this', 'word[-...","[NON_NER, NON_NER, NON_NER, NON_NER, NON_NER, ..."
1,"[{'bias': 1.0, 'word.lower()': 'that', 'word[-...","[NON_NER, NON_NER, NON_NER, NON_NER, NON_NER, ..."
2,"[{'bias': 1.0, 'word.lower()': 'new', 'word[-3...","[NON_NER, NON_NER, NON_NER, NON_NER, NON_NER, ..."
3,"[{'bias': 1.0, 'word.lower()': 'on', 'word[-3:...","[NON_NER, DATE, DATE, NON_NER, DATE, NON_NER, ..."
4,"[{'bias': 1.0, 'word.lower()': 'i', 'word[-3:]...","[NON_NER, NON_NER, NON_NER, NON_NER, NON_NER, ..."


In [15]:
train = data.iloc[:int(len(data)*0.7)]
test = data.iloc[int(len(data)*0.7):]

In [16]:
X_train = train['feat'].to_list()
y_train = train['label'].to_list()
X_test = test['feat'].to_list()
y_test = test['label'].to_list()

In [18]:
import time

In [19]:
start = time.time()
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

print(f"Training Time : {(time.time() - start)/60}")

Training Time : 31.180336101849875


In [20]:
crf.tagger_.dump(filename="crf_tagger.txt")

In [21]:
labels = list(crf.classes_)
# labels.remove('O')
labels.sort()
labels

['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NON_NER',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [43]:
# import joblib

# # save
# joblib.dump(crf, "model.pkl") 

# # load
# clf2 = joblib.load("model.pkl")

# # clf2.predict(X[0:1])

In [23]:
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [42]:
y_pred = crf.predict(X_test)

In [29]:
len(y_pred)

252949

In [48]:
# y_test[:10]

In [40]:
y_test[0][:10]

['NON_NER',
 'PERSON',
 'NON_NER',
 'NON_NER',
 'NON_NER',
 'NON_NER',
 'NON_NER',
 'NON_NER',
 'NON_NER',
 'NON_NER']

In [50]:
from sklearn.preprocessing import MultiLabelBinarizer
# y = [[2, 3, 4], [2], [0, 1, 3], [0, 1, 2, 3, 4], [0, 1, 2]]
trans = MultiLabelBinarizer().fit(y_test)
y_trans = trans.transform(y_test)
y_pred_trans = trans.transform(y_pred)

In [46]:
y_trans[:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0]])

In [54]:
y_test_expand = []
for i in y_test: y_test_expand.extend(i) 

In [55]:
y_pred_expand = []
for i in y_pred: y_pred_expand.extend(i) 

In [56]:
print(classification_report(
    y_test_expand, y_pred_expand, labels=labels, digits=3
))

/packages/apps/jupyter/2023-10-09/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/packages/apps/jupyter/2023-10-09/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    CARDINAL      0.592     0.425     0.495     83652
        DATE      0.722     0.713     0.717    158171
       EVENT      0.000     0.000     0.000       760
         FAC      0.000     0.000     0.000      2276
         GPE      0.861     0.172     0.287     74287
    LANGUAGE      0.000     0.000     0.000       492
         LAW      0.000     0.000     0.000       338
         LOC      0.000     0.000     0.000      4313
       MONEY      0.642     0.772     0.701     31868
     NON_NER      0.956     0.993     0.974   9000608
        NORP      0.810     0.568     0.668     84288
     ORDINAL      0.000     0.000     0.000      7157
         ORG      0.740     0.221     0.341    112706
     PERCENT      0.000     0.000     0.000     15230
      PERSON      0.748     0.121     0.209    136683
     PRODUCT      0.000     0.000     0.000       593
    QUANTITY      0.000     0.000     0.000      2361
        TIME      0.000    

/packages/apps/jupyter/2023-10-09/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.3.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /packages/apps/jupyter/2023-10-09/lib/python3.11/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: eli5, mapclassify
